In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Conversation Memory

In langrila, message and response are serializable, so you can store the conversation history whereever the json object can be stored. Currently support: 

- InMemory
- Pickle (local file)
- JSON (local file)
- S3
- Azure Cosmos DB

If you build multi-agent, the conversation history in the top-level orchestrator is stored while the one in the subagent isn't stored. However, the top-level conversation history is input into subagent as well. 

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import (
    Agent,
    InMemoryConversationMemory,
    JSONConversationMemory,
    PickleConversationMemory,
)
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.memory.cosmos import CosmosConversationMemory
from langrila.memory.s3 import S3ConversationMemory
from langrila.openai import OpenAIClient

# Instantiate client wrapper

In [4]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(api_key_env_name="GEMINI_API_KEY")

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(api_key_env_name="ANTHROPIC_API_KEY")

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

# Define tools

Using dummy tools

In [5]:
import random
from typing import Literal


def power_disco_ball(power: bool) -> bool:
    """
    Powers the spinning dissco ball.

    Parameters
    ----------
    power : bool
        Whether to power the disco ball or not.

    Returns
    ----------
    bool
        Whether the disco ball is spinning or not.
    """
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music() -> str:
    """
    Turn on the music. The genre, BPM, and volume are randomly selected.

    Returns
    ----------
    str
        A message indicating that the music is starting.
    """
    music_genre = random.choice(["rock", "pop", "jazz", "classical", "hip-hop"])
    volume = random.uniform(0.2, 1.0)
    bpm = 120
    return f"Starting music! Genre: {music_genre}, BPM: {bpm}, Volume: {volume}"


def change_bpm(new_bpm: int) -> int:
    """
    Change the BPM of the music.

    Parameters
    ----------
    new_bpm : int
        The new BPM to set the music.

    Returns
    ----------
    int
        The new BPM of the music.
    """
    return f"Changing BPM to {new_bpm}"


def change_music(
    genre: Literal["rock", "pop", "jazz", "classical", "hip-hop"],
    bpm: int,
) -> str:
    """
    Change the music genre and BPM.

    Parameters
    ----------
    genre : str
        The genre of music to play. Should be one of "rock", "pop", "jazz", "classical", or "hip-hop".
    bpm : int
        The BPM of the music.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    return f"Changing music to {genre} with BPM {bpm}"


def turn_light_on() -> str:
    """
    Turn the lights on.

    Returns
    ----------
    str
        A message indicating that the lights are turning on.
    """
    brightness = random.uniform(0.5, 1.0)
    return "Lights are now on! Brightness: {:.2f}".format(brightness)


def dim_lights(brightness: float) -> bool:
    """
    Dim the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully dimmed.
    """
    return f"Lights are now set to {brightness}"


def adjust_volume(volume: float) -> bool:
    """
    Adjust the volume of the music.

    Parameters
    ----------
    volume : float
        The volume level to set the music. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the volume was successfully adjusted.
    """
    return f"Volume is now set to {volume}"


def stop_music() -> str:
    """
    Stop the music.

    Returns
    ----------
    str
        A message indicating that the music is stopping.
    """
    return "Stopping music!"


def stop_disco_ball() -> str:
    """
    Stop the disco ball power and spinning.

    Returns
    ----------
    str
        A message indicating that the disco ball is stopping.
    """
    return "Stopping disco ball! Powered off and stopped spinning."


def adjust_lights(brightness: float) -> bool:
    """
    Adjust the brightness of the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully brightened.
    """
    return f"Lights are now set to {brightness}"

# Multi-Agent

## Response schema

In [6]:
from enum import Enum

from pydantic import BaseModel, Field


class DiscoBallSchema(BaseModel):
    power: bool = Field(..., description="Whether to power the disco ball.")
    spinning: bool = Field(..., description="Whether the disco ball is spinning.")


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class MusicSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music to play.",
    )
    bpm: int = Field(
        ...,
        description="The BPM of the music.",
        ge=60,
        le=180,
    )
    volume: float = Field(
        ...,
        description="The volume level to set the music to.",
        ge=0,
        le=1,
    )


class LightsSchema(BaseModel):
    brightness: float = Field(
        ...,
        description="The brightness level to set the lights to.",
        ge=0,
        le=1,
    )


class ResponseSchema(BaseModel):
    disco_ball: DiscoBallSchema = Field(..., description="The disco ball settings.")
    music: MusicSchema = Field(..., description="The music settings.")
    lights: LightsSchema = Field(..., description="The lights settings.")

## Task specific agent.

In [7]:
lights_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=[turn_light_on, adjust_lights],
    response_schema_as_tool=LightsSchema,
)

disco_ball_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[power_disco_ball, stop_disco_ball],
    max_tokens=500,
    response_schema_as_tool=DiscoBallSchema,
    planning=True,  # Planning is enabled for sub agents.
)


music_power_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[start_music],
)

music_control_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=[change_music, adjust_volume, change_bpm],
)

## Orchestrator agent

In [8]:
# Orchestrator agent as a subagent
music_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    subagents=[music_power_agent, music_control_agent],
    max_tokens=500,
    response_schema_as_tool=MusicSchema,
)

# InMemoryConversationMemory

In [14]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=InMemoryConversationMemory(),
    planning=True,
)

In [15]:
prompt = "Turn this place into a party mood."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:56:31] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 20:56:

In [16]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>, 'bpm': 120, 'volume': 0.728},
 'lights': {'brightness': 0.8}}

In [17]:
list(response.usage.items())

[('disco_ball_agent',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=1365, output_tokens=107)),
 ('lights_agent',
  Usage(model_name='gemini-2.0-flash-exp', prompt_tokens=466, output_tokens=14)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=2590, output_tokens=348)),
 ('music_power_agent',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=174, output_tokens=41)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=3341, output_tokens=416))]

Next turn prompt

In [18]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:57:02] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 20:57:02

In [19]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>, 'bpm': 120, 'volume': 0.728},
 'lights': {'brightness': 0.8}}

Load history.

In [20]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)

If you want to input conversation history while you don't want to store the response into the history, please set `store_conversation` as `False` in the `AgentConfig`.

In [21]:
from langrila import AgentConfig

# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=InMemoryConversationMemory(),
    planning=True,
    agent_config=AgentConfig(store_conversation=False),  # Disable storing conversation history
)

# PickleConversationMemory

`PickleConversationMemory` keeps the history as the local pickle file.

In [22]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=PickleConversationMemory("./conversation.pkl"),
    planning=True,
)

In [23]:
prompt = "Turn this place into a party mood."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:58:35] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 20:58:

In [24]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:59:08] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 20:59:08

In [25]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>, 'bpm': 130, 'volume': 0.75},
 'lights': {'brightness': 0.8}}

Load history

In [26]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)

# JSONConversationMemory

`JSONConversationMemory` stores the history into the local json file.

In [27]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=JSONConversationMemory("./conversation.json"),
    planning=True,
)

In [28]:
prompt = "Turn this place into a party mood."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:59:32] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 20:59:

In [29]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 21:00:06] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 21:00:06

In [30]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.hip_hop: 'hip-hop'>,
  'bpm': 128,
  'volume': 0.8},
 'lights': {'brightness': 0.8}}

Load history

In [31]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)

# S3ConversationMemory

`S3ConversationMemory` allows you to store the history into the AWS S3 bucket.

Acknowledgement: [@kun432](https://github.com/kun432)

In [34]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=S3ConversationMemory(
        bucket="langrila-test",
        object_key="sample_conversation.json",  # "PREFIX/OBJECT_KEY" for using prefix
        aws_access_key_id_env_name="AWS_ACCESS_KEY",  # # env variable name, as needed
        aws_secret_access_key_env_name="AWS_SECRET_KEY",  # # env variable name, as needed
    ),
    planning=True,
)

In [35]:
prompt = "Turn this place into a party mood."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 21:02:33] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 21:02:

In [36]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 21:03:04] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 21:03:04

In [37]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.rock: 'rock'>, 'bpm': 140, 'volume': 0.75},
 'lights': {'brightness': 0.8}}

In [38]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)

# CosmosConversationMemory

`CosmosConversationMemory` is the module to store history to the Azure Cosmos DB. Please create Cosmos DB for your azure subscription in advance.

Acknowledgement: [@rioriost](https://github.com/rioriost) for base implementation.

In [39]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=CosmosConversationMemory(
        endpoint_env_name="COSMOS_ENDPOINT",  # env variable name
        key_env_name="COSMOS_KEY",  # env variable name
        db_env_name="COSMOS_DB_NAME",  # env variable name
        container_name="test",
        item_name="sample_conversation",
    ),
    planning=True,
)

In [40]:
prompt = "Turn this place into a party mood."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 21:03:57] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 21:03:

In [41]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 21:04:30] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 21:04:30

In [42]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>, 'bpm': 130, 'volume': 0.8},
 'lights': {'brightness': 0.8}}

In [43]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)